In [1]:
import pandas as pd
import numpy as np
import tarfile
import re
%reload_ext version_information
np.__version__, pd.__version__

('1.16.4', '0.24.2')

<h3>2a) Einlesen und Entpacken des tar.gz Files mit tarfile</h3>

In [2]:
tar = tarfile.open("20news-18828.tar.gz")
#tar.extractall() #gefährlich, entpackt direkt alles
#tar.close()

<h3> 2b) Speichern der Beiträge der Newsgroups <I>alt.atheism, comp.graphics, sci.space </I>
    und <I>talk.religion.misc</I> als Strings in ein Array</h3>

In [3]:
i = 0
for tarinfo in tar:
    print(tarinfo.name, "is", tarinfo.size, "bytes in size and is ", end="")
    if tarinfo.isreg():
        print("a regular file.")
    elif tarinfo.isdir():
        print("a directory.")
    else:
        print("something else.")
        
    i+=1
    if i == 20:
        break


20news-18828 is 0 bytes in size and is a directory.
20news-18828/alt.atheism is 0 bytes in size and is a directory.
20news-18828/alt.atheism/51203 is 1372 bytes in size and is a regular file.
20news-18828/alt.atheism/51277 is 1514 bytes in size and is a regular file.
20news-18828/alt.atheism/53192 is 1752 bytes in size and is a regular file.
20news-18828/alt.atheism/53222 is 1991 bytes in size and is a regular file.
20news-18828/alt.atheism/51283 is 2615 bytes in size and is a regular file.
20news-18828/alt.atheism/53759 is 577 bytes in size and is a regular file.
20news-18828/alt.atheism/53225 is 753 bytes in size and is a regular file.
20news-18828/alt.atheism/53098 is 1200 bytes in size and is a regular file.
20news-18828/alt.atheism/51275 is 2386 bytes in size and is a regular file.
20news-18828/alt.atheism/53601 is 892 bytes in size and is a regular file.
20news-18828/alt.atheism/54173 is 1587 bytes in size and is a regular file.
20news-18828/alt.atheism/53334 is 1045 bytes in siz

In [4]:
i = 0
groups = ["20news-18828/alt.atheism", "20news-18828/comp.graphics", "20news-18828/sci.space", "20news-18828/talk.religion.misc"]
data = []
group_vect = []
group_cnt = []
group = None
cnt = 0
needed = False
for tarinfo in tar:
    if tarinfo.isdir() and tarinfo.name in groups:
        if needed:
            group_cnt.append(cnt)
            cnt = 0
        needed = True
        group = tarinfo.name
    elif tarinfo.isreg() and needed:
        data.append(open(tarinfo.name).read())
        group_vect.append(group)
        cnt += 1
    elif tarinfo.isdir() and tarinfo.name not in groups:
        if needed:
            group_cnt.append(cnt)
            cnt = 0
        needed = False
        group = None
if needed:
    group_cnt.append(cnt)

Kontrolle: Es müssten jetzt 3387 Strings
im Speicher sein

In [5]:
tar.close()
np.shape(data), np.shape(group_vect)

((3387,), (3387,))

In [54]:
group_cnt

[799, 973, 987, 628]

Entfernen der ersten zwei Zeilen, also dem Newsgroup-Header, aus den Dateien

In [7]:
def strip_header(text):
    _before, _blankline, after = text.partition ('\n\n')
    return after


In [8]:
data = [strip_header(text) for text in data]

<h3>2c) Aufteilen der Strings in Worte(Token) und speichern der verschiedenen Worte über alle Texte</h3>

Berechnen eines Merkmalsvektors für jeden Text, der für jedes Wort des Vokabulars seine Häufigkeit innerhalb des Texts
enthält.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
merk_vect = count_vect.fit_transform(data).toarray()
merk_vect.shape

(3387, 41777)

Kontrolle: Es sollten 41777 verschiedene Worte und 3387 Texte vorhanden sein

In [10]:
# Anzahl der Worte im ersten Text
np.sum(merk_vect[0])

198

<h3>d. Verwenden Sie die ersten 60% der Daten als Trainingsdatensatz, die restlichen als Testdatensatz. Trainieren Sie damit einen multinomialen naiven Bayes-Klassifikator. Bestimmen
Sie den Anteil korrekter Klassifikationen auf Ihren Trainings- und Testdaten. Wie gut
generalisiert Ihr Klassifikator?</h3>

In [31]:
train_ath_cnt = int(np.floor(group_cnt[0] * 0.6))
test_ath_cnt = int(np.ceil(group_cnt[0] * 0.4))

train_gr_cnt = int(np.floor(group_cnt[1] * 0.6))
test_gr_cnt = int(np.ceil(group_cnt[1] * 0.4))

train_sp_cnt = int(np.floor(group_cnt[2] * 0.6))
test_sp_cnt = int(np.ceil(group_cnt[2] * 0.4))

train_rel_cnt = int(np.floor(group_cnt[3] * 0.6))
test_rel_cnt = int(np.ceil(group_cnt[3] * 0.4))

Aufteilen der Daten in die einzelnen Klassen

In [75]:
atheism = merk_vect[0 : group_cnt[0]]

graphic = merk_vect[group_cnt[0] : group_cnt[0] + group_cnt[1]]

space = merk_vect[group_cnt[0] + group_cnt[1] : group_cnt[0] + group_cnt[1] + group_cnt[2]]

religion = merk_vect[group_cnt[0] + group_cnt[1] + group_cnt[2] : np.sum(group_cnt)]

Prüfen ob die Zahlen stimmen

In [56]:
atheism.shape, graphic.shape, space.shape, religion.shape, group_cnt

((799, 41777), (973, 41777), (987, 41777), (628, 41777), [799, 973, 987, 628])

Prüfen, ob sich atheism und graphic überschneiden

In [49]:
set(atheism[-1] == graphic[0])

{False, True}

Splitten der Daten in Trainings- und Testdaten

In [60]:
train_atheism = atheism[0 : train_ath_cnt]
test_atheism = atheism[train_ath_cnt : train_ath_cnt + test_ath_cnt]

train_graphic = graphic[0 : train_gr_cnt]
test_graphic = graphic[train_gr_cnt : train_gr_cnt + test_gr_cnt]

train_space = space[0 : train_sp_cnt]
test_space = space[train_sp_cnt : train_sp_cnt + test_sp_cnt]

train_religion = religion[0 : train_rel_cnt]
test_religion = religion[train_rel_cnt : train_rel_cnt + test_rel_cnt]

In [61]:
train_atheism.shape[0] + test_atheism.shape[0]

799

In [63]:
train_ath_cnt + train_gr_cnt + train_sp_cnt + train_rel_cnt, test_ath_cnt + test_gr_cnt + test_sp_cnt + test_rel_cnt

(2030, 1357)

Für jedes einzigartige Wort wird für jede Klasse die Wahrscheinlichkeit dieses Wortes in dieser Klasse berechnet.

In [76]:
atheism_prob = train_atheism.sum(axis=0) / train_atheism.sum()
graphic_prob = train_graphic.sum(axis=0) / train_graphic.sum()
space_prob = train_space.sum(axis=0) / train_space.sum()
religion_prob = train_religion.sum(axis=0) / train_religion.sum()

In [77]:
atheism_prob.sum()

1.0

In [12]:
%version_information numpy, pandas, sklearn

Software versions
Python 3.7.3 64bit [MSC v.1915 64 bit (AMD64)]
IPython 7.6.1
OS Windows 10 10.0.18362 SP0
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
Tue Nov 19 12:09:57 2019 Mitteleuropäische Zeit